In [1]:
kubectl delete jobs --all 2>&1 | tail 
kubectl delete pods --all 2>&1 | tail
kubectl delete services shuffle || true

job.batch "reader90" deleted
job.batch "reader91" deleted
job.batch "reader92" deleted
job.batch "reader93" deleted
job.batch "reader94" deleted
job.batch "reader95" deleted
job.batch "reader96" deleted
job.batch "reader97" deleted
job.batch "reader98" deleted
job.batch "reader99" deleted
pod "writer10" deleted
pod "writer11" deleted
pod "writer12" deleted
pod "writer13" deleted
pod "writer14" deleted
pod "writer15" deleted
pod "writer16" deleted
pod "writer17" deleted
pod "writer18" deleted
pod "writer19" deleted
service "shuffle" deleted


In [2]:
cat > kubetpl.yaml <<'EOF'
image: gcr.io/research-191823/simple
memory: 8G
app: shuffle
subdomain: shuffle
EOF

# Name resolution between pods requires a headless service with an app selector to be run; selected pods will get name-resolved.
kubetpl service | kubectl apply -f -

service/shuffle created


In [3]:
src=gs://lpr-g1000-words/*.tgz
dst=gs://lpr-g1000-words2
out=$dst/g1000-words
image=gcr.io/research-191823/simple

gsutil mb $dst || true

echo -n src=; gsutil ls $src | wc -l
echo -n dst=; gsutil ls $dst | wc -l
echo -n out=; gsutil ls $out/* | wc -l

Creating gs://lpr-g1000-words2/...
ServiceException: 409 Bucket lpr-g1000-words2 already exists.
src=980
dst=0
out=CommandException: One or more URLs matched no objects.
0


In [4]:
for index in {00..19}; do
kubetpl pod -C 2 -M 16G -n writer$index -c "
hostname; date
tarsplit zpull://0.0.0.0:10000 -s 1e9 -n 1e6 -z -o '|gsutil cp - $dst/TEMP-$index-{shard:06d}.tgz'
hostname; date
" | kubectl apply -f -
done

pod/writer00 created
pod/writer01 created
pod/writer02 created
pod/writer03 created
pod/writer04 created
pod/writer05 created
pod/writer06 created
pod/writer07 created
pod/writer08 created
pod/writer09 created
pod/writer10 created
pod/writer11 created
pod/writer12 created
pod/writer13 created
pod/writer14 created
pod/writer15 created
pod/writer16 created
pod/writer17 created
pod/writer18 created
pod/writer19 created


In [5]:
gsutil ls $src | sort -R | cat -n |
while read index shard; do
kubetpl job -M 8G -L jobgroup=readers -I $image -n reader$index -c "
gsutil cat $shard | tarscat --shuffle 10000 -o 'zpush://writer{00..19}.shuffle:10000'
" | kubectl apply -f -
done

job.batch/reader1 created
job.batch/reader2 created
job.batch/reader3 created
job.batch/reader4 created
job.batch/reader5 created
job.batch/reader6 created
job.batch/reader7 created
job.batch/reader8 created
job.batch/reader9 created
job.batch/reader10 created
job.batch/reader11 created
job.batch/reader12 created
job.batch/reader13 created
job.batch/reader14 created
job.batch/reader15 created
job.batch/reader16 created
job.batch/reader17 created
job.batch/reader18 created
job.batch/reader19 created
job.batch/reader20 created
job.batch/reader21 created
job.batch/reader22 created
job.batch/reader23 created
job.batch/reader24 created
job.batch/reader25 created
job.batch/reader26 created
job.batch/reader27 created
job.batch/reader28 created
job.batch/reader29 created
job.batch/reader30 created
job.batch/reader31 created
job.batch/reader32 created
job.batch/reader33 created
job.batch/reader34 created
job.batch/reader35 created
job.batch/reader36 created
job.batch/reader37 created
job.batch/

In [6]:
sleep 60
kubectl logs job.batch/reader1 | tail

# send <tarproclib.zcom.MultiWriter object at 0x7f5e94301828> 0
# send <tarproclib.zcom.MultiWriter object at 0x7f5e94301828> 10000
# send <tarproclib.zcom.MultiWriter object at 0x7f5e94301828> 20000
# send <tarproclib.zcom.MultiWriter object at 0x7f5e94301828> 30000
# send <tarproclib.zcom.MultiWriter object at 0x7f5e94301828> 40000
# send <tarproclib.zcom.MultiWriter object at 0x7f5e94301828> 50000
# send <tarproclib.zcom.MultiWriter object at 0x7f5e94301828> 60000
# send <tarproclib.zcom.MultiWriter object at 0x7f5e94301828> 70000
# send <tarproclib.zcom.MultiWriter object at 0x7f5e94301828> 80000
# send <tarproclib.zcom.MultiWriter object at 0x7f5e94301828> 90000


In [7]:
kubectl logs writer01 | tail

# recv <tarproclib.zcom.Connection object at 0x7f56805ed6a0> 620000             
# recv <tarproclib.zcom.Connection object at 0x7f56805ed6a0> 630000             
# writing |gsutil cp - gs://lpr-g1000-words2/TEMP-01-000006.tgz (635566, 6000023089)
Copying from <STDIN>...   0.0 B]      0.0 B/s                                   
/ [1 files][    0.0 B/    0.0 B]      0.0 B/s                                   
Operation completed over 1 objects.                                              
# recv <tarproclib.zcom.Connection object at 0x7f56805ed6a0> 640000
# recv <tarproclib.zcom.Connection object at 0x7f56805ed6a0> 650000             
# recv <tarproclib.zcom.Connection object at 0x7f56805ed6a0> 660000             
# recv <tarproclib.zcom.Connection object at 0x7f56805ed6a0> 670000             


In [8]:
date

Mon Nov 18 08:58:29 PST 2019


In [9]:
kubectl wait jobs -l jobgroup=readers --for=condition=complete --timeout=-1s

job.batch/reader1 condition met
job.batch/reader10 condition met
job.batch/reader100 condition met
job.batch/reader101 condition met
job.batch/reader102 condition met
job.batch/reader103 condition met
job.batch/reader104 condition met
job.batch/reader105 condition met
job.batch/reader106 condition met
job.batch/reader107 condition met
job.batch/reader108 condition met
job.batch/reader109 condition met
job.batch/reader11 condition met
job.batch/reader110 condition met
job.batch/reader111 condition met
job.batch/reader112 condition met
job.batch/reader113 condition met
job.batch/reader114 condition met
job.batch/reader115 condition met
job.batch/reader116 condition met
job.batch/reader117 condition met
job.batch/reader118 condition met
job.batch/reader119 condition met
job.batch/reader12 condition met
job.batch/reader120 condition met
job.batch/reader121 condition met
job.batch/reader122 condition met
job.batch/reader123 condition met
job.batch/reader124 condition met
job.batch/reader125

In [10]:
date

Mon Nov 18 09:16:07 PST 2019


In [11]:
echo -n src=; gsutil ls $src | wc -l
echo -n dst=; gsutil ls $dst | wc -l
echo -n out=; gsutil ls $out/* | wc -l

src=980
dst=260
out=CommandException: One or more URLs matched no objects.
0


In [12]:
sleep 3
kubectl get pods | grep Running

writer00          1/1     Running     0          37m
writer01          1/1     Running     0          37m
writer02          1/1     Running     0          37m
writer03          1/1     Running     0          37m
writer04          1/1     Running     0          37m
writer05          1/1     Running     0          37m
writer06          1/1     Running     0          37m
writer07          1/1     Running     0          37m
writer08          1/1     Running     0          37m
writer09          1/1     Running     0          37m
writer10          1/1     Running     0          37m
writer11          1/1     Running     0          37m
writer12          1/1     Running     0          37m
writer13          1/1     Running     0          37m
writer14          1/1     Running     0          37m
writer15          1/1     Running     0          37m
writer16          1/1     Running     0          37m
writer17          1/1     Running     0          37m
writer18          1/1     Running     0       

In [13]:
kubectl exec writer00 -- tarscat --nodata --eof -o 'zpush://writer{00..19}.shuffle:10000'

command terminated with exit code 137


: 137

In [14]:
sleep 3
kubectl get pods | grep Running

: 1

In [15]:
gsutil ls $dst/TEMP-*.tgz | awk '{printf("gsutil mv %s '$out'-%06d.tgz\n", $1, count+0); count+=1}' | bash -x

+ gsutil mv gs://lpr-g1000-words2/TEMP-00-000000.tgz gs://lpr-g1000-words2/cdip-words-000000.tgz
Copying gs://lpr-g1000-words2/TEMP-00-000000.tgz [Content-Type=application/octet-stream]...
Removing gs://lpr-g1000-words2/TEMP-00-000000.tgz...                            
-
Operation completed over 1 objects/949.8 MiB.                                    
+ gsutil mv gs://lpr-g1000-words2/TEMP-00-000001.tgz gs://lpr-g1000-words2/cdip-words-000001.tgz
Copying gs://lpr-g1000-words2/TEMP-00-000001.tgz [Content-Type=application/octet-stream]...
Removing gs://lpr-g1000-words2/TEMP-00-000001.tgz...                            
-
Operation completed over 1 objects/949.7 MiB.                                    
+ gsutil mv gs://lpr-g1000-words2/TEMP-00-000002.tgz gs://lpr-g1000-words2/cdip-words-000002.tgz
Copying gs://lpr-g1000-words2/TEMP-00-000002.tgz [Content-Type=application/octet-stream]...
Removing gs://lpr-g1000-words2/TEMP-00-000002.tgz...                            

Operation completed o

In [16]:
echo -n src=; gsutil ls $src | wc -l
echo -n dst=; gsutil ls $dst | wc -l
echo -n out=; gsutil ls $out* | wc -l

src=980
dst=260
out=260
